In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re

pd.options.display.max_columns = None

**Pair Programming Limpieza II**

Antes de seguir, recordamos las preguntas que nos planteamos al principio del _pair programming_** de EDA para dirigir nuestro análisis.**:

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?

- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?

- Independientemente de la edad, sufren los hombres más ataques que las mujeres?

- ¿En qué mes ocurren más ataques?

- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?

- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex.

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra.

Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉. Para esto trabajaremos con el fichero generado en Limpieza I.

1. Columna de species:

Si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna.

Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

Imaginemos que el valor de una celda es el siguiente:
​
'White shark, 3.5 m'
​
Tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
Un patrón que podriamos usar es: 
​
patron_blanco = r".*[Ww](hite|HITE).*" (esto podría ser así porque puede estar en mayúsculas o en minúsculas.)
​
De la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar".

Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [4]:
df = pd.read_csv("df_interesantes.csv", index_col = 0)
df.head()

,YEAR,COUNTRY,SEX_,AGE,SPECIES_,fecha,FATALIDAD
0,2018,usa,F,57,White shark,NaN,N
1,2018,brazil,M,18,Tiger shark,NaN,N
2,2018,usa,M,15,"Bull shark, 6'",NaN,N
3,2018,australia,M,32,Grey reef shark,NaN,N
4,2018,england,M,21,Invalid incident,NaN,N


In [5]:
patron_white = r".*[Ww](hite|HITE).*"
patron_tiger = r".*[Tt](iger|IGER).*"
patron_grey = r".*[Gg](rey|REY).*"
patron_lemon = r".*[Ll](emon|EMON).*"
patron_bull = r".*[Bb](ull|ULL).*"

In [6]:
def especies(col):
        patron_white = re.findall (".*[Ww](hite|HITE).*", str(col))
        patron_tiger = re.findall (".*[Tt](iger|IGER).*", str(col))
        patron_grey = re.findall (".*[Gg](rey|REY).*", str(col))
        patron_lemon = re.findall (".*[Ll](emon|EMON).*", str(col))
        patron_bull = re.findall (".*[Bb](ull|ULL).*", str(col))

        if len(patron_white) > 0:
            return "white"
        elif len(patron_tiger) > 0:
            return "Tiger"
        elif len(patron_grey) > 0:
            return "Grey"
        elif len (patron_lemon) > 0:
            return "Lemon"
        elif len (patron_bull) > 0:
            return "Bull"
        else:
            return "Desconocido"
    

In [7]:
df ["nueva_especie"] = (df["SPECIES_"].apply(especies))
df

,YEAR,COUNTRY,SEX_,AGE,SPECIES_,fecha,FATALIDAD,nueva_especie
0,2018,usa,F,57,White shark,NaN,N,white
1,2018,brazil,M,18,Tiger shark,NaN,N,Tiger
2,2018,usa,M,15,"Bull shark, 6'",NaN,N,Bull
3,2018,australia,M,32,Grey reef shark,NaN,N,Grey
4,2018,england,M,21,Invalid incident,NaN,N,Desconocido
...,...,...,...,...,...,...,...,...
1649,0,south africa,NaN,NaN,NaN,NaN,N,Desconocido
1657,0,fiji,NaN,NaN,NaN,NaN,Y,Desconocido
1661,0,asia?,NaN,NaN,NaN,NaN,N,Desconocido
1670,0,panama,NaN,NaN,NaN,NaN,Y,Desconocido



2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:

- Edad en formato string.

- Edades separadas por &, or, to, >

- Edades con ?


Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:

- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.

Puede que os salga un error similar a este:

TypeError: expected string or bytes-like object

Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

df['nombre_columna'] = df['nombre_columna'].astype(str)

Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.

- Por último cambiad el tipo de la columna de string a integer.

In [8]:
df

,YEAR,COUNTRY,SEX_,AGE,SPECIES_,fecha,FATALIDAD,nueva_especie
0,2018,usa,F,57,White shark,NaN,N,white
1,2018,brazil,M,18,Tiger shark,NaN,N,Tiger
2,2018,usa,M,15,"Bull shark, 6'",NaN,N,Bull
3,2018,australia,M,32,Grey reef shark,NaN,N,Grey
4,2018,england,M,21,Invalid incident,NaN,N,Desconocido
...,...,...,...,...,...,...,...,...
1649,0,south africa,NaN,NaN,NaN,NaN,N,Desconocido
1657,0,fiji,NaN,NaN,NaN,NaN,Y,Desconocido
1661,0,asia?,NaN,NaN,NaN,NaN,N,Desconocido
1670,0,panama,NaN,NaN,NaN,NaN,Y,Desconocido


In [9]:
patron_numeros = r".*([0-9]+).*"

In [10]:
df['AGE'] = df['AGE'].astype(object)

In [11]:
def numeros (col):
    patron_numeros = re.findall (r".*([0-9]+).*", str(col))

    if len(patron_numeros) > 0:
        return str(patron_numeros [0])
    

In [12]:
df["edad_limpia"] = df["AGE"].apply(numeros)

In [19]:
df

,YEAR,COUNTRY,SEX_,AGE,SPECIES_,fecha,FATALIDAD,nueva_especie,edad_limpia
1133,1976,usa,M,29,"White shark, 5.5 m [18']",NaN,N,white,9
917,1993,spain,M,69,1.2 m [4'] shark,NaN,N,Desconocido,9
1468,1931,australia,M,39,Shark involvement prior to death unconfirmed,NaN,N,Desconocido,9
541,2005,usa,M,30,White shark,NaN,N,white,0
1345,1960,australia,M,13,Bull shark,NaN,N,Bull,3
...,...,...,...,...,...,...,...,...,...
1185,1969,usa,M,13,4' shark,NaN,N,Desconocido,3
1457,1936,australia,M,14,White shark,NaN,N,white,4
992,1989,australia,M,30,Grey nurse shark,NaN,N,Grey,0
426,2008,brazil,M,14,Bull shark,NaN,N,Bull,4


In [24]:
edades_divididas = df.explode("edad_limpia", ignore_index=False)
edades_divididas.sample(50)

,YEAR,COUNTRY,SEX_,AGE,SPECIES_,fecha,FATALIDAD,nueva_especie,edad_limpia
358,2010,australia,M,18,"White shark, 2.5m",NaN,N,white,8
674,2002,turks & caicos,F,41,1.8 m to 2.1 m [6' to 7'] Caribbean Reef Shark,NaN,N,Desconocido,1
1257,1963,south africa,F,20,"Zambesi shark, 1.8 m [6']",NaN,N,Desconocido,0
567,2005,usa,M,25,Shark involvement not confirmed,NaN,N,Desconocido,5
454,2007,usa,M,22,3' to 4' shark,NaN,N,Desconocido,2
1245,1964,mexico,M,19,1.8 to 2.4 m [6' to 8'] shark,NaN,N,Desconocido,9
828,1998,mozambique,F,35,3 m [10'] shark,NaN,N,Desconocido,5
387,2009,usa,M,31,2' to 3' shark,NaN,N,Desconocido,1
1545,77,NaN,NaN,NaN,NaN,NaN,Y,Desconocido,None
1307,1961,australia,M,23,"Grey nurse shark, 2.4 m [8']",NaN,N,Grey,3


In [25]:
edades_divididas.dtypes

YEAR              int64
COUNTRY          object
SEX_             object
AGE              object
SPECIES_         object
fecha            object
FATALIDAD        object
nueva_especie    object
edad_limpia      object
dtype: object

In [38]:
edades_divididas["edad_limpia"]= pd.to_numeric (edades_divididas["edad_limpia"])
edades_divididas

,YEAR,COUNTRY,SEX_,AGE,SPECIES_,fecha,FATALIDAD,nueva_especie,edad_limpia
0,2018,usa,F,57,White shark,NaN,N,white,7.0
1,2018,brazil,M,18,Tiger shark,NaN,N,Tiger,8.0
2,2018,usa,M,15,"Bull shark, 6'",NaN,N,Bull,5.0
3,2018,australia,M,32,Grey reef shark,NaN,N,Grey,2.0
4,2018,england,M,21,Invalid incident,NaN,N,Desconocido,1.0
...,...,...,...,...,...,...,...,...,...
1649,0,south africa,NaN,NaN,NaN,NaN,N,Desconocido,NaN
1657,0,fiji,NaN,NaN,NaN,NaN,Y,Desconocido,NaN
1661,0,asia?,NaN,NaN,NaN,NaN,N,Desconocido,NaN
1670,0,panama,NaN,NaN,NaN,NaN,Y,Desconocido,NaN


3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [39]:
# pasamos al siguiente pair con las edades en float por recomendacion de los profesores.
edades_divididas.to_csv("limpieza_dos.csv")
